In [1]:
from bs4 import BeautifulSoup
# from selenium import webdriver
import requests
# import time
from datetime import datetime

In [2]:
url = 'https://ie.trustpilot.com/review/bywinona.com?page=3'# 'https://www.trustpilot.com/review/joinmidi.com?page=3' #'https://ie.trustpilot.com/review/bywinona.com?page='
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>
<html lang="en-IE"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/favicon.ico" rel="shortcut icon" type="image/x-icon"/><link href="/manifest.json" rel="manifest"/><meta content="Trustpilot" name="application-name"/><meta content="#1c1c1c" name="theme-color"/><link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/><link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/><link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/><link color="#00b67a" href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/safari-pinned-tab.svg" rel="mask-icon"/><meta content="Trustpilot" name="apple-mobile-web-app-title"/><meta content="#1c1c1c" name="msap

In [4]:
pagination = soup.find('nav', {'aria-label': 'Pagination'})
last_page = pagination.find_all('a')[-2].text.strip()
last_page

'14'

In [8]:
reviews = soup.find_all('section', class_='styles_reviewContentwrapper__zH_9M')
reviews, len(reviews), type(reviews)

([<section aria-disabled="false" class="styles_reviewContentwrapper__zH_9M"><div class="styles_reviewHeader__iU9Px" data-service-review-rating="5"><div class="star-rating_starRating__4rrcf star-rating_medium__iN6Ty"><img alt="Rated 5 out of 5 stars" src="https://cdn.trustpilot.net/brand-assets/4.1.0/stars/stars-5.svg"/></div><div class="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_datesWrapper__RCEKH"><time class="" data-service-review-date-time-ago="true" datetime="2024-08-17T17:52:55.000Z">Aug 17, 2024</time></div></div><div aria-hidden="false" class="styles_reviewContent__0Q2Tg" data-review-content="true"><a class="link_internal__7XN06 typography_appearance-default__AAY17 typography_color-inherit__TlgPO link_link__IZzHN link_notUnderlined__szqki" data-review-title-typography="true" href="/reviews/66c0c7579b31fb4018abd207" rel="nofollow" target="_self"><h2 class="typography_heading-s__f7029 typography_appearance-default__AAY17" data-service-review-title-typogra

In [3]:
comments = soup.find_all('div', class_='styles_reviewCardInner__EwDq2')
comments, len(comments), type(comments)

([<div class="styles_reviewCardInner__EwDq2"><aside aria-label="Info for Kimberly M" class="styles_consumerInfoWrapper__KP3Ra"><div class="styles_consumerDetailsWrapper__p2wdr"><div class="avatar_avatar__hmBp6 avatar_pink__GYN9_" data-consumer-avatar="true" style="width:44px;min-width:44px;height:44px;min-height:44px"><span class="typography_heading-xs__jSwUz typography_appearance-default__AAY17 typography_disableResponsiveSizing__OuNP7 avatar_avatarName__ehkAr">KM</span></div><a class="link_internal__7XN06 link_wrapper__5ZJEx styles_consumerDetails__ZFieb" data-consumer-profile-link="true" href="/users/66d6fbdb29e32d86c393cd4e" name="consumer-profile" rel="nofollow" target="_self"><span class="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17" data-consumer-name-typography="true">Kimberly M</span><div class="styles_consumerExtraDetails__fxS4S" data-consumer-reviews-count="1"><span class="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l" data-consumer-revie

# (info by, reviewer, location, verified, date, review, date of experience, reply by)

In [5]:
all_comments = []
for review in comments:
    aside = review.find('aside')
    info_by = aside.get('aria-label')

    span_reviewer = review.find('span', class_='typography_heading-xxs__QKBS8 typography_appearance-default__AAY17')
    reviewer = span_reviewer.text.strip() if span_reviewer else None

    spans = aside.find_all('span')
    location = spans[-1].get_text()

    div_verified = soup.find('div', class_='styles_reviewLabels__nHPUT styles_reviewLabels__Ym2vM')
    verified = True if div_verified.find_all('span')[-1].text.strip() == 'Verified' else False # type: ignore 

    title = review.find('h2', class_='typography_heading-s__f7029').text.strip()
    paragraph = review.find('p', class_='typography_body-l__KUYFJ').text.strip()
    date_of_experience = review.find('p', class_='typography_body-m__xgxZ_').text.strip().split(': ')[1]
    time_posted = review.find('time').text.strip()
    datetime_posted = review.find('time').get('datetime')
    rated_img = soup.find('div', class_='star-rating_starRating__4rrcf star-rating_medium__iN6Ty').find('img') # type: ignore
    rated_alt = rated_img.get('alt') if rated_img else None # type: ignore 
    rated = float(rated_alt.split()[1])
    reply = review.find('div', class_='styles_wrapper__ib2L5')
    if reply:
        reply_author = reply.find('p', {'data-service-review-business-reply-title-typography': 'true'}).text.strip()
        reply_date = reply.find('time',class_='typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_replyDate__Iem0_').text.strip()
        reply_datetime_posted = reply.find('time').get('datetime')
        reply_paragraph = reply.find('p', {'data-service-review-business-reply-text-typography': 'true'}).text.strip()
    else:
        reply_author = None
        reply_date = None
        reply_datetime_posted = None
        reply_paragraph = None


    all_comments.append({
        "date" : datetime.now().strftime("%Y-%m-%d %H:%M:%S"), #.strftime('%Y-%m-%d'),
        'info by': info_by,
        'author_name': reviewer,
        'author_location': location,
        'verified': verified,
        'rated': rated,
        'title': title,
        'time_posted': (time_posted, datetime_posted),
        'paragraph': paragraph,
        'date_of_experience': date_of_experience,
        'reply_author': reply_author,
        'reply_date': (reply_date, reply_datetime_posted),
        'reply_paragraph': reply_paragraph
    })

all_comments

[{'date': '2024-09-12 19:19:28',
  'info by': 'Info for Kimberly M',
  'author_name': 'Kimberly M',
  'author_location': 'US',
  'verified': True,
  'rated': 4.5,
  'title': 'I’m a work in progress…',
  'time_posted': ('3 Sept 2024', '2024-09-03T14:06:53.000Z'),
  'paragraph': 'I’ve been dealing with menopause symptoms for a few years, my symptoms were so bad, I felt helpless! I started on HRT about 3 months ago and I feel about 50% better, but I’m working with my doctor to adjust my medication, I’m looking forward to seeing how the new medication will help. I would recommend Winona to anyone, it has been a pleasant experience.',
  'date_of_experience': '03 September 2024',
  'reply_author': 'Reply from By Winona',
  'reply_date': ('3 Sept 2024', '2024-09-03T20:53:00.000Z'),
  'reply_paragraph': "We are so happy to support you, Kimberly!A lot of women report the same feelings of helplessness and despair regarding their menopause symptoms, but we are here to help!It sounds like you've b